### Before you begin

To check whether the code you've written is correct, we'll use **automark**. For this, we created for each of you an account with the username being your student number.

In [ ]:
import automark as am

# fill in you student number as your username
am.configure(username='0')

# to check your progress, you can run this function
am.get_progress()

## Settings things up for the excerises

In this assignment, we work with an example database consisting of the following six tables:

<img src="schema.png" alt="Schema of the Database" width="900"/>

### DuckDB

We will query this database with [DuckDB](https://duckdb.org/), an embeddable analytical database management system, developed by the [CWI Amsterdam](https://www.cwi.nl/research/groups/database-architectures). 

DuckDB is designed to be easy to install and allows to seamlessly work in a jupyter notebook.

DuckDB also provides a [SQL Introduction](https://duckdb.org/docs/sql/introduction) that you can refer to when working on this assignment.

In [ ]:
import duckdb

We connect to an in-memory database.

In [ ]:
con = duckdb.connect(database=':memory:')

Next, we create all the relations using SQL

In [ ]:
# Customers Table
con.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id char(10) NOT NULL, 
    firstname varchar(32) default NULL, 
    lastname varchar(32) default NULL, 
    city varchar(32) default NULL,
    address varchar(128) default NULL, 
    email varchar(128) default NULL, 
    
    PRIMARY KEY (customer_id) )
''')

# Laptops Table
con.execute('''
CREATE TABLE IF NOT EXISTS laptops (
    model char(4) NOT NULL default '',
    speed double default NULL, 
    ram int default NULL, 
    hd int default NULL, 
    screen double default NULL, 
    price double default NULL, 
    
    PRIMARY KEY  (model) )
''')

# PCs Table
con.execute('''
CREATE TABLE IF NOT EXISTS pcs (
    model char(4) NOT NULL, 
    speed double NOT NULL, 
    ram int NOT NULL, 
    hd int NOT NULL, 
    price double NOT NULL, 
    
    PRIMARY KEY  (model) )
''')

# Printers Table
con.execute('''CREATE TABLE IF NOT EXISTS printers (
    model char(4) NOT NULL default '', 
    color varchar(5) default NULL, 
    type varchar(10) default NULL,
    price double default NULL,

    PRIMARY KEY  (model) )
''')

# Products Table
con.execute('''CREATE TABLE IF NOT EXISTS products ( 
    maker char(1) default NULL,
    model char(4) NOT NULL default '',
    type varchar(10) default NULL,

PRIMARY KEY  (model) )
''')

# Sales Table
con.execute('''CREATE TABLE IF NOT EXISTS sales ( 
    customer_id char(10) NOT NULL default '', 
    model char(4) NOT NULL default '',   
    quantity int default NULL,   
    day date NOT NULL default '0000-00-00', 
    paid double default NULL,   
    type_of_payment varchar(32) default NULL,   
    
    PRIMARY KEY  (customer_id,model,day) )
''')

And fill the tables with some example data

In [ ]:
# Customers Table
con.execute('''
INSERT INTO customers (customer_id, firstname, lastname, city, address, email)
VALUES 
    ('1122334455', 'Ann', 'O''Brien', 'Rotterdam', '1 Jervis St.', 'ann@uva.nl'), 
    ('1231231231', 'John', 'Doe', 'Amsterdam', NULL, NULL), 
    ('1234567890', 'Maya', 'Ramanath', 'Diemen', 'Dalsteindreef 3002', NULL),
    ('9876543210', 'Ji', 'Zhang', 'Utrecht', 'Jaarbeursplien 24', 'jack@ucc.ie'), 
    ('9999999999', 'Norah', 'Jones', 'Amsterdam', 'Passheuvelweg 34', 'nj@yahoo.com');
''')

# Laptops Table
con.execute('''
INSERT INTO laptops (model, speed, ram, hd, screen, price) 
VALUES 
    ('2001', 2, 2048, 240, 20.1, 3673),
    ('2002', 1.73, 1024, 80, 17, 949), 
    ('2003', 1.8, 512, 60, 15.4, 549), 
    ('2004', 2, 512, 60, 13.3, 1150),
    ('2005', 2.16, 1024, 120, 17, 2500), 
    ('2006', 2, 2048, 80, 15.4, 1700), 
    ('2007', 1.83, 1024, 120, 13.3, 1429),
    ('2008', 1.6, 1024, 100, 15.4, 900), 
    ('2009', 1.6, 512, 80, 14.1, 680), 
    ('2010', 2, 2048, 160, 15.4, 2300);
''')

# PCs Table
con.execute('''
INSERT INTO pcs (model, speed, ram, hd, price) 
VALUES 
    ('1001', 2.66, 1024, 250, 2114),
    ('1002', 2.1, 512, 250, 995), 
    ('1003', 1.42, 512, 80, 478), 
    ('1004', 2.8, 1024, 250, 649), 
    ('1005', 3.2, 512, 250, 630), 
    ('1006', 3.2, 1024, 320, 1049), 
    ('1007', 2.2, 1024, 200, 510), 
    ('1008', 2.2, 2048, 250, 770), 
    ('1009', 2, 1024, 250, 650), 
    ('1010', 2.8, 2048, 300, 770),
    ('1011', 1.86, 2048, 160, 959), 
    ('1012', 2.8, 1024, 160, 649), 
    ('1013', 3.06, 512, 80, 529);
''')

# Printers Table
con.execute('''
INSERT INTO printers (model, color, type, price) 
VALUES 
    ('3001', 'TRUE', 'ink-jet', 99),
    ('3002', 'FALSE', 'laser', 239), 
    ('3003', 'TRUE', 'laser', 899), 
    ('3004', 'TRUE', 'ink-jet', 120),
    ('3005', 'FALSE', 'laser', 120), 
    ('3006', 'TRUE', 'ink-jet', 100), 
    ('3007', 'TRUE', 'laser', 200);
''')

# Products Table
con.execute('''
INSERT INTO products (maker, model, type) 
VALUES 
    ('A', '1001', 'pc'), 
    ('A', '1002', 'pc'),
    ('A', '1003', 'pc'), 
    ('B', '1004', 'pc'), 
    ('B', '1005', 'pc'), 
    ('B', '1006', 'pc'), 
    ('C', '1007', 'pc'),
    ('D', '1008', 'pc'), 
    ('D', '1009', 'pc'), 
    ('D', '1010', 'pc'), 
    ('E', '1011', 'pc'), 
    ('E', '1012', 'pc'),
    ('E', '1013', 'pc'), 
    ('E', '2001', 'laptop'), 
    ('E', '2002', 'laptop'), 
    ('E', '2003', 'laptop'),
    ('A', '2004', 'laptop'), 
    ('A', '2005', 'laptop'), 
    ('A', '2006', 'laptop'), 
    ('B', '2007', 'laptop'),
    ('F', '2008', 'laptop'), 
    ('F', '2009', 'laptop'), 
    ('G', '2010', 'laptop'), 
    ('E', '3001', 'printer'), 
    ('E', '3002', 'printer'), 
    ('E', '3003', 'printer'), 
    ('D', '3004', 'printer'), 
    ('D', '3005', 'printer'),
    ('H', '3006', 'printer'), 
    ('H', '3007', 'printer'); 
''')

# Sales Table
con.execute('''
INSERT INTO sales (customer_id, model, quantity, day, paid, type_of_payment) 
VALUES 
    ('1122334455', '2010', 1, '2020-12-19', 2300, 'mastercard credit'), 
    ('1122334455', '3001', 1, '2020-12-18', 99, 'cash'), 
    ('1231231231', '2002', 2, '2020-12-19', 1898, 'visa credit'), 
    ('1231231231', '3002', 1, '2020-12-18', 239, 'cash'), 
    ('1234567890', '1001', 1, '2020-12-20', 1902.6, 'mastercard credit'),
    ('9876543210', '1007', 1, '2020-12-17', 510, 'visa debit'), 
    ('9876543210', '1007', 3, '2020-12-19', 1530, 'visa debit'), 
    ('9876543210', '2002', 1, '2020-12-17', 949, 'visa debit'), 
    ('9999999999', '1007', 1, '2020-12-20', 459, 'visa credit'), 
    ('9999999999', '3007', 2, '2020-12-20', 360, 'visa credit');
''')

Now, we can query the database using SQL and convert the results to pandas dataframes to look at them:

In [ ]:
products = con.execute("SELECT * FROM products").fetchdf()
products

In [ ]:
pcs = con.execute("SELECT * FROM pcs").fetchdf()
pcs

In [ ]:
printers = con.execute("SELECT * FROM printers").fetchdf()
printers

In [ ]:
sales = con.execute("SELECT * FROM sales").fetchdf()
sales

In [ ]:
customers = con.execute("SELECT * FROM customers").fetchdf()
customers

In [ ]:
laptops = con.execute("SELECT * FROM laptops").fetchdf()
laptops

The following helper function allows us to run queries on the database:

In [ ]:
def execute_local(query):
    con = duckdb.connect(database=':memory:', read_only=False)
    con.register('products', products)
    con.register('customers', customers)
    con.register('sales',sales)
    con.register('laptops',laptops)
    con.register('pcs',pcs)
    
    result = con.execute(query).fetchdf()
    
    return result

This code is required to display things in a nicer way.

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Part A: Basic SQL Select Queries

In the following, we ask you to write a set of SQL queries for querying the database.


### Task 1. List only the model number of all products made by maker B.

In [ ]:
def a1a_t1_all_model_number_b():
    """List only the model number of all products made by maker B.
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__model__|
|---|
|1004|
|1005|
|1006|
|2007|


In [ ]:
execute_local(a1a_t1_all_model_number_b())

In [ ]:
am.test_student_function(a1a_t1_all_model_number_b)

### Task 2. List the model numbers of all products not made by maker B.

In [ ]:
def a1a_t2_all_model_number_not_b():
    """List the model numbers of all products not made by maker B."""
    #################
    ### YOUR CODE ###
    #################
    query = ""

    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__model__|
|---|
|1001|
|1002|
|1003|
|1007|
|1008|
|1009|
|1010|
|1011|
|1012|
|1013|
|2001|
|2002|
|2003|
|2004|
|2005|
|2006|
|2008|
|2009|
|2010|
|3001|
|3002|
|3003|
|3004|
|3005|
|3006|
|3007|

In [ ]:
execute_local(a1a_t2_all_model_number_not_b())

In [ ]:
am.test_student_function(a1a_t2_all_model_number_not_b)

###  Task 3. Return the average speed of all PCs with speed >=3.00.

In [ ]:
def a1a_t3_avg_PCs_speed():
    """Return the average speed all PCs with speed >= 3.00"""
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__avg(speed)__|
|---|
|3.153333|

In [ ]:
execute_local(a1a_t3_avg_PCs_speed())

In [ ]:
am.test_student_function(a1a_t3_avg_PCs_speed)

###  Task 4. Return the price of the most expensive color laser printer.

In [ ]:
def a1a_t4_max_printer_price():
    """Return the price of the most expensive color laser printer"""
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__max(price)__|
|---|
|899.0|

In [ ]:
execute_local(a1a_t4_max_printer_price())

In [ ]:
am.test_student_function(a1a_t4_max_printer_price)

###  Task 5. Return the minimum amount paid by customers who used a visa card (debit or credit) to purchase a product.

In [ ]:
def a1a_t5_min_spending_visa():
    """Return the minimum amount paid by customers who used a visa card (debit or credit) to purchase a product"""
    #################
    ### YOUR CODE ###
    #################
    query = ""
   
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__min(paid)__|
|---|
|360.0|

In [ ]:
execute_local(a1a_t5_min_spending_visa())

In [ ]:
am.test_student_function(a1a_t5_min_spending_visa)

### Task 6. Find the `customer_id` of customers who have the letter 'e' either in their first name or in their last name.

In [ ]:
def a1a_t6_customer_id():
    """Find the customer_id of customers who have the letter 'e' either in their first name 
       or in their last name"""
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__customer_id__|
|---|
|1122334455|
|1231231231|
|9999999999|

In [ ]:
execute_local(a1a_t6_customer_id())

In [ ]:
am.test_student_function(a1a_t6_customer_id)

### Task 7. Assume all prices in the Laptops table are in Euro. List the model numbers of all laptops with ram at least 1024. For each model, list also its price in USD. Assume that 1 USD = 0.85 EURO. Name the price column 'price (USD)'.

In [ ]:
def a1a_t7_highest_price_USD():
    """Assume all prices in the table Laptops are in Euro. List the prices of laptops with at least 1024 ram. 
    You should return the price in USD in a column called 'price (USD)'. Assume that 1 USD = 0.85 EURO. 
    Name the price column 'price (USD)'.
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""
 
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__model__|__price (USD)__|
|---|---|
|2001|4321.176471|
|2002|1116.470588|
|2005|2941.176471|
|2006|2000.000000|
|2007|1681.176471|
|2008|1058.823529|
|2010|2705.882353|

In [ ]:
execute_local(a1a_t7_highest_price_USD())

In [ ]:
am.test_student_function(a1a_t7_highest_price_USD)

## Part B: SQL queries involving joins and grouping

### Task 1

### Task 1a. Return a list of makers that make more than four different products.

In [ ]:
def a1b_t1a_at_least_four_products():
    """Return a list of makers that make more than four different products.
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__maker__|
|---|
|A|
|D|
|E|

In [ ]:
execute_local(a1b_t1a_at_least_four_products())

In [ ]:
am.test_student_function(a1b_t1a_at_least_four_products)

### Task 1b. List all the laptop model numbers that have a speed greater than 1.7 in descending order.

In [ ]:
def a1b_t1b_decending_speed_laptop():
    """ List all the laptop model numbers that have a speed greater than 1.7 in descending order.
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__model__|
|---|
|2005|
|2001|
|2004|
|2006|
|2010|
|2007|
|2003|
|2002|

In [ ]:
execute_local(a1b_t1b_decending_speed_laptop())

In [ ]:
am.test_student_function(a1b_t1b_decending_speed_laptop)

### Task 1c. List all first names of customers in an ascending order based on the number of purchases made by customers with that first name.

In [ ]:
def a1b_t1c_customer_number_purchase():
    """ List firstnames of customers in an ascending order 
    based on the number of purchases made by customers with this firstname.
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__firstname__|
|---|
|Maya|
|Ann|
|John|
|Nora|
|Ji|

In [ ]:
execute_local(a1b_t1c_customer_number_purchase())

In [ ]:
am.test_student_function(a1b_t1c_customer_number_purchase)

### Task 1d. List all the makers (with only one entry per maker) who make PCs with RAM greater than 1500.

In [ ]:
def a1b_t1d_maker_with_ram():
    """ List all the makers (with only one entry per maker) who make PCs with RAM greater than 1500.
    """
    #################
    ### YOUR CODE ###
    #################   
    query = ""

    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__maker__|
|---|
|D|
|E|

In [ ]:
execute_local(a1b_t1d_maker_with_ram())

In [ ]:
am.test_student_function(a1b_t1d_maker_with_ram)

### Task 2

Write SQL SELECT statements for performing the following queries. Use aggregations and group by.

### Task 2a. Find the city and the average amount of money spent by customers in each city. Name the column for the amount 'avg_spend'.

In [ ]:
def a1b_t2a_avg_money_spent():
    """ Find the city and the average amount of money spent by customers in each city. 
        Name the column for the amount 'avg_spend'
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""

    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__city__|__avg_spend__|
|---|---|
|Rotterdam|1199.500000|
|Amsterdam|739.000000|
|Diemen|1902.600000|
|Utrecht|996.333333|

In [ ]:
execute_local(a1b_t2a_avg_money_spent())

In [ ]:
am.test_student_function(a1b_t2a_avg_money_spent)

###  Task 2b. Find the maximum price for both color and non-color printers. Name the column for the maximum price 'max_price'.

In [ ]:
def a1b_t2b_maximum_price_printer():
    """ Find the maximum price for both a color and non-color printers.
        Name the column for the maximum price 'max_price'.
    """
    #################
    ### YOUR CODE ###
    #################
    query = ""
    
    return query

You can test your query by running it on our local database via the `execute_local` function and inspecting the results.

We expect the following result:

|__color__|__max_price__|
|---|---|
|TRUE|899.0|
|FALSE|239.0|

In [ ]:
execute_local(a1b_t2b_maximum_price_printer())

In [ ]:
am.test_student_function(a1b_t2b_maximum_price_printer)